# Approche non modale de la stabilité dans un écoulement de Poiseuille

## Table des matières

- Introduction et motivation
- Implémentation et test de la svd de $e^{Mt}$, calculée à partir des modes propres de $M$
    - Commençons par implémenter la svd pour un instant $t$ fixé 
    - Test de la svd sur un intervalle de temps
    - Soucis avec l'emploi de la matrice $M$
- Calcul détaillé de l'augmentation maximale pour un (Re,$\alpha$) donné
    - Efficacité de la stratégie de division récursive pour la recherche d'un maximum
- Implémentation et test de la svd de $e^{Mt}$, calculée avec Runge et Kutta
    - Implémentation du calcul de l'exponentielle
    - Test de la svd
- Implémentation sur gpu avec Runge et Kutta
- Comparaison des vitesses d'exécution entre numpy et tensorflow

## Introduction et motivation

Dans les notebooks précédents, nous avions étudié l'équation d'Orr-Sommerfeld que nous avions discrétisée et ramenée à un problème aux valeurs propres généralisé $A\hat v = Bc \hat v$ en considérant des modes propres de la forme $\hat v = v(y)\, e^{i\alpha (x-ct)}$.
 
Nous pouvons ramener ce problème aux valeurs propres à une équation d'évolution pour les modes propres. En effet, nous observons que

$$ \partial_t \hat v = -i\alpha c \hat v   \qquad   B^{-1}A \hat v = c \hat v $$

Et donc 

$$ \partial_t \hat v = M \hat v   \qquad M = -i\alpha B^{-1}A$$

Dans le notebook précédant, nous avions discuté de la stabilité de l'écoulement en observant la partie imaginaire des valeurs propres de l'opérateur $M$. Dans ce notebook, nous allons effectuer une analyse plus détaillée de l'évolution des perturbations. Nous nous limiterons plus à étudier le comportement des modes propres, car une telle analyse est incomplète étant donné la nature non linéaire des équations de Navier-Stokes. 

L'inconvénient de l'étude modale de la stabilité vient de la non orthogonalité des vecteurs propres de l'opérateur $M$. En effet, la matrice $M$ n'est pas hermitienne et il est donc possible que ces vecteurs propres soient non orthogonaux. Cela a pour conséquence qu'une combinaison linéaire de deux vecteurs propres, dont la norme diminue dans le temps, peut voir sa norme grandir pendant un temps fini. Un problème avec cette croissance temporaire de certaines perturbations se pose lorsque celle-ci devient trop importante. L'approximation linéaire des équation de Navier-Stokes sur laquelle repose l'équation d'Orr-Sommerfeld n'est alors plus valable et la perturbation considérée, composée que de mode stables *selon l'équation d'Orr-Sommerfeld*, pourrait devenir instable.

Il est donc nécessaire de déterminer quelles sont les croissances maximales que l'on peut obtenir avec des combinaisons linéaires des modes stables afin de vérifier si celles-ci ne sont pas trop grandes pour engendrer des instablilités. Ce travail s'effectue typiquement par une décomposition en valeurs singulières (svd), qui va chercher l'élongaltion et la direction d'élongation maximale de la boule unité sous une transformation linéaire.

## Implémentation et test de la svd de $e^{Mt}$, calculée à partir des modes propres de $M$

### Commençons par implémenter la svd pour un instant $t$ fixé 

In [ ]:
import numpy,scipy,scipy.linalg,time,pandas
import matplotlib.pyplot as pyplot
from matplotlib import rcParams, cm
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

Importons les fonctions que nous avons défini afin de construire les opérateurs nécessaires.

In [ ]:
import operateurs

In [ ]:
N = 401 
alpha = 0.6
Re = 10000
t = 1

[A,B] = operateurs.buildAB_forward(Re, alpha, N)

Calculons l'exponentielle $e^{Mt}$ à partir de la décomposition en valeurs et vecteurs propres de M. Lors de cette phase de construction, nous affichons les variables pour pouvoir vérifier les résultats.

In [ ]:
def expMt_modes(A,B,alpha,t):

    [c,vecp] = scipy.linalg.eig(A,B)
    c = -1j*alpha*c # pour avoir les valeurs propres de M = -i*alpha*B^-1*A
    vecp_inv = numpy.linalg.inv(vecp)

    D = scipy.eye(N-4) * numpy.exp(c*t)
    return numpy.dot( numpy.dot( vecp , D) , vecp_inv )

In [ ]:
expMt = expMt_modes(A,B,alpha,t)
print(expMt[:2,:2])

Nous faisons ensuite le calcul de la svd

In [ ]:
u,s,v = scipy.linalg.svd(expMt)
print(s[:5])

### Test de la svd sur un intervalle de temps

Nous implémentons le calcul de la svd pour plusieurs points de temps dans des fichiers séparés que nous importons ci-dessous.

In [ ]:
import svd

In [ ]:
N = 401
alpha = 0.6
Re = 10000
t_vec = numpy.linspace(0,100,50)

[A,B] = operateurs.buildAB_forward(Re, alpha, N)

start = time.time()
s = svd.svd_scipy_exp_eig_A_B(A, B, alpha, t_vec, False)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))

# valeurs singulières max pour chaque t
print("Valeurs singulières: ")
print(s[:,0])

In [ ]:
pyplot.figure(figsize=(8,6));
pyplot.plot(t_vec, s[:,0], '.');
pyplot.title('Valeur singulière maximale\n au cours du temps');
pyplot.xlabel('$t$');
pyplot.ylabel('Valeur singulière maximale');
pyplot.grid();
pyplot.show();

### Soucis avec l'emploi de la matrice $M$

Remarquons que nous avons effectué la décomposition en modes propres de $M$ à partir du problème généralisé défini par $A$ et $B$. Nous évitons de calculer l'exponentielle de $M$ à partir de sa décomposition directe de $M$ car cette dernière n'est pas correctement réalisée par l'ordinateur, probablement à cause de l'inversion de la matrice $B$ qui comporte des nombres très petits. Les décompositions respectives avec $M$ d'une part et $A,B$ de l'autre sont réalisées ci-dessous.

In [ ]:
N = 401
alpha = 0.6
Re = 10000
t_vec = numpy.linspace(0,0.5,10)

[A,B] = operateurs.buildAB_forward(Re, alpha, N)
M = numpy.dot(numpy.linalg.inv(B), A) 

# Décomposition en valeur et vecteur propres avec A et B
[c,vecp] = scipy.linalg.eig(A,B)

pyplot.figure(figsize=(8,6));
pyplot.plot(numpy.real(c), numpy.imag(c), '.');
pyplot.title('valeurs propres avec A et B');
pyplot.xlabel('Re(c)');
pyplot.ylabel('Im(c)');
pyplot.grid();
pyplot.show();

# Décomposition en valeur et vecteur propres de M
[c,vecp] = scipy.linalg.eig(M)

pyplot.figure(figsize=(8,6));
pyplot.plot(numpy.real(c), numpy.imag(c), '.');
pyplot.title('valeurs propres avec M');
pyplot.xlabel('Re(c)');
pyplot.ylabel('Im(c)');
pyplot.grid();
pyplot.show();

## Calcul détaillé de l'augmentation maximale pour un (Re,$\alpha$) donné

Le problème auquel nous faisons face est la recherche du maximum de la fonction $f(t)$ que défini le calcul de plus grande valeur singulière de $e^{Mt}$ en chaque $t$.

Pour accélérer la recherche de ce maximum, nous allons exploiter la monotonie apparente de la fonction $f$ avant et après le maximum et employer la stratégie récursive suivante:
- Commencer avec un intervalle suspecté de contenir le maximum
- Diviser l'intervalle en plusieurs segments
- Evaluer $f$ en les jonctions entre ces segments
- Trouver le point de jonction $t_\star$ qui maximise $f$
- Prendre comme nouvel intervalle l'union des deux segments de part et d'autre de $t_\star$
- Recommencer jusqu'à atteintre la précision voulue

Cette méthode est implémentée dans un fichier que nous importons ci-dessous.

Remarque: Selon la finesse de la grille, la fonction $f$ peut subir une augmentation locale entre $t=0$ et $t=1$. Il faut comparer le résultat de cette recherche avec le graphique de $f$ réalisé plus haut. Il serait intéressant de systématiquement vérifier l'augmentation maximale avec plus de points entre 0 et 1.

In [ ]:
import svd_division_recursive as rd

In [ ]:
N = 401
alpha = 0.6
Re = 10000
tmin = 0
tmax = 100
target_prec = 0.01

[A,B] = operateurs.buildAB_forward(Re, alpha, N)

start = time.time()
s_max,t_max,err_tmax = rd.svd_division_recursive_A_B(svd.svd_scipy_exp_eig_A_B, A, B, alpha, tmin, tmax, target_prec, 6)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))

print("Résultats: ")
print("Valeur singulière maximale:",s_max)
print("Instant de perturbation maximale:",t_max, "+/-", err_tmax)

La valeur trouvée semble bien correspondre à ce que nous avions obtenu précédemment.

### Efficacité de la stratégie de division récursive pour la recherche d'un maximum

Nous montrons ci-dessous l'efficacité de plusieurs stratégies de recherche de maximum en calculant la précision que nous pouvons atteindre en effectuant au maximum $N$ calculs de svd. 

La précision considérée est la taille de l'intervalle contenant le maximum après $N$ calculs par rapport à sa taille initiale.

Les stratégies que nous testons sont les suivantes:
- Stratégie "force brute": Evaluation en pleins de points répartits uniformément dans l'intervalle initialement choisi. La précision diminue à peu près comme l'inverse de N
- Stratégie "division récursive", expliquée plus haut, que nous testons pour plusieurs nombres de segments $d$.

In [ ]:
N = range(2,200)

# incertitudes avec N calculs de svd
def incertinv(N):
    return 2/(N-1)

def incertexp(N,d):
    it = int(N/(d+1))  # une itération est composée de d+1 évaluations de la fonction à maximiser
    return (2/d)**it

vinv = numpy.vectorize(incertinv)
vexp = numpy.vectorize(incertexp)

pyplot.figure(figsize=(8,6));
pyplot.loglog(N, vinv(N), '.-', label='inverse');
pyplot.loglog(N, vexp(N,3), '.-', label='exp d=3');
pyplot.loglog(N, vexp(N,6), '.-', label='exp d=6');
pyplot.loglog(N, vexp(N,10), '.-', label='exp d=10');
pyplot.title('Incertitude suivant le nombre d\'évaluations');
pyplot.xlabel('N');
pyplot.ylabel('incertitude');
pyplot.legend(loc=3);
pyplot.grid();
pyplot.show();

Nous constatons sans surprise que la stratégie récursive est la plus efficace, avec un maximum d'efficacité autour d'une division en 6 segments. C'est donc la stratégie que nous avons utilisé dès que nous en avons l'occasion.

## Implémentation et test de la svd de $e^{Mt}$, calculée avec Runge et Kutta

### Implémentation du calcul de l'exponentielle

L'inconvénient avec le calcul de $e^{Mt}$ à partir des valeurs et vecteurs propres de $M$ est que cette méthode est limitée au cas où $M$ est indépendante du temps. Si $M$ varie dans le temps, en particulier si le profil de vitesse $U(y,t)$ dépend du temps, alors la décomposition en modes propres de $M(t)$ est à refaire pour chaque instant $t$.

Heureusement, il existe une manière plus efficace de calculer l'exponentielle $e^{Mt}$ sans refaire systématiquement la décomposition en modes propres. Cette méthode consiste à intégrer avec l'algorithme de Runge et Kutta une équation différentielle dont $e^{Mt}$ est la solution. Il facile de construire une telle équation en remarquant que:

$$ \frac{d}{dt} e^{Mt} = M e^{Mt} $$

Nous voyons donc que $e^{Mt}$ est solution du problème de Cauchy suivant:

$$ \dot L(t) = M L(t) \qquad L(0) = \mathbb{1}$$

Nous réécrivons l'équation différentielle sous la forme

$$ \dot L(t) = f(t,L)  \qquad f(t,L) = M(t)L $$

Nous allons dans ce qui suit implémenter cette méthode et commencer par la tester avec le profil $U(y,t) = 1-y^2$ indépendant du temps pour pouvoir comparer les résultat avec la méthode utilisant les modes propres de $M$. L'intégrateur choisi sera Runge et Kutta d'ordre 4.

Nous importons une version adaptée du code de construction des opérateurs:

In [ ]:
import operateurs_t

In [ ]:
def U_statique(y,t):
    return 1-y**2

def f(t,L,Re,alpha,N):
    
    [A,B] = operateurs_t.buildAB_forward(Re, alpha, N, U_statique, t)
    B_inv = numpy.linalg.inv(B)
    M = -1j*alpha*numpy.dot(B_inv,A)
    
    return numpy.dot(M,L)

Nous commençons par calculer l'exponentielle $e^{Mt}$ en un temps $t$ donné.

In [ ]:
N = 401
alpha = 0.6
Re = 10000
t_max = 10
dt = 0.1

nt = int(t_max/dt)+1

t = 0
L = scipy.eye(N-4)

for i in range(1,nt+1):
    
    # La matrice L=exp(Mt) est avancé de dt dans le temps avec RK4
    k1 = dt*f(t,L,Re,alpha,N)
    k2 = dt*f(t+dt/2,L+k1/2,Re,alpha,N)
    k3 = dt*f(t+dt/2,L+k2/2,Re,alpha,N)
    k4 = dt*f(t+dt,L+k3,Re,alpha,N)
    
    t = t+dt
    L = L + 1/6*(k1+2*k2+2*k3+k4)

Nous pouvons ensuite comparer le résultat avec celui donné par la méthode aux modes propres implémentée plus haut.

In [ ]:
print(t)
print("L_RK4",L[:2,:2])

L_modes = expMt_modes(A,B,alpha,t)
print("L_modes",L_modes[:2,:2])

Nous remarquons que nous pouvons obtenir des résultats très proches (indistinguables avec le nombre de décimales affiché par défaut) pour des $dt$ suffisamment petits. Typiquement $\leq 0.1$ dans notre example. Cependant, l'exponentielle calculée par RK4 diverge rapidement pour des pas de temps supérieur à une certaine valeur critique, située entre $0.1$ et $0.2$ dans notre cas. Il doit probablement y avoir une certaine condition de convergence que nous pourrions imposer au pas de temps $dt$.

### Test de la svd

Ci-dessous, nous réalisons la svd pour plusieurs temps en utilisant RK4. La méthode est implémentée dans le fichier "svd_t" que nous importons ci-dessous.

Au passage, vérifions que les résultats pour deux discrétisations proches donnent des résultats similaires.

In [ ]:
import svd_t

In [ ]:
def U_statique(y,t):
    return 1-y**2

In [ ]:
N = 401
alpha = 0.6
Re = 10000
t_max = 100
dt = 0.05
step = 20 # nous réalisons le calcul de la svd qu'une fois sur step

start = time.time()
st,t_vec = svd_t.svd_scipy_RK4(Re,alpha,N,t_max,dt,step,U_statique)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))

In [ ]:
N = 601
alpha = 0.6
Re = 10000
t_max = 100
dt = 0.05
step = 20 # nous réalisons le calcul de la svd qu'une fois sur step

start = time.time()
st2,t_vec2 = svd_t.svd_scipy_RK4(Re,alpha,N,t_max,dt,step,U_statique)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))

In [ ]:
pyplot.figure(figsize=(8,6));
pyplot.plot(t_vec, st[:,0], '.',label="N=401");
pyplot.plot(t_vec2, st2[:,0], '.',label="N=601");
pyplot.title('Valeur singulière maximale\n au cours du temps');
pyplot.xlabel('$t$');
pyplot.ylabel('Valeur singulière maximale');
pyplot.grid();
pyplot.show();

In [ ]:
print(st[:,0])

Nous vérifions que la courbe de la valeur singulière maximale avec le temps est cohérente avec celle utilisant la décomposition en modes propres. Les courbes pour les deux discrétisations testée sont bien proches.

Ci-dessous, un morceau de code pour sauver le résultat du calcul dans un fichier.

In [ ]:
dataSet = list(zip(t_vec2[:,0], st2[:,0]))
df = pandas.DataFrame(data = dataSet, columns=['Temps (s)', 'Valeur singulière Max'])
df.to_csv('dataNumpy/data.csv',index=False,header=True)

## Implémentation sur gpu avec Runge et Kutta

Nous allons dans cette section utiliser le module "tensorflow" pour effectuer le calcul de l'exponentielle sur une carte graphique (gpu). L'implémentation des méthodes est réalisée dans le fichier que nous importons ci-dessous.

In [ ]:
import svd_t_tf

In [ ]:
N = 401
alpha = 0.6
Re = 10000
t_max = 100
dt = 0.1
step = 10 # nous réalisons le calcul de la svd qu'une fois sur step

start = time.time()
st,t_vec = svd_t_tf.svd_tf_RK4(Re,alpha,N,t_max,dt,step,U_statique)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))

In [ ]:
pyplot.figure(figsize=(8,6));
pyplot.plot(t_vec, st[:,0], '.');
pyplot.title('Valeur singulière maximale\n au cours du temps');
pyplot.xlabel('$t$');
pyplot.ylabel('Valeur singulière maximale');
pyplot.grid();
pyplot.show();

## Comparaison des vitesses d'exécution entre numpy et tensorflow

Nous réalisons ici la comparaison entre les vitesses d'exécutions des codes numpy (cpu) et tensorflow (gpu) sur une machine aux caractéristiques suivantes:

- carte graphique: Nvidia GTX 1080 Ti
- processeur: Intel(R) Xeon(R) CPU E5-2690
- OS: Ubuntu 16.04

Comparons pour $N=401$:

In [ ]:
N = 401
alpha = 0.6
Re = 10000
t_max = 0.5
dt = 0.1 # 0.5/0.1 = 5 pas de temps
step = 10000 # pas de svd pour le test

#numpy
start = time.time()
st,t_vec = svd_t.svd_scipy_RK4(Re,alpha,N,t_max,dt,step,U_statique)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))

#tensorflow
start = time.time()
st,t_vec = svd_t_tf.svd_tf_RK4(Re,alpha,N,t_max,dt,step,U_statique)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))

Comparons pour $N=1001$:

In [ ]:
N = 1001
alpha = 0.6
Re = 10000
t_max = 0.1
dt = 0.02 # 0.1/0.02 = 5 pas de temps
step = 10000 # pas de svd pour le test

#numpy
start = time.time()
st,t_vec = svd_t.svd_scipy_RK4(Re,alpha,N,t_max,dt,step,U_statique)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))

#tensorflow
start = time.time()
st,t_vec = svd_t_tf.svd_tf_RK4(Re,alpha,N,t_max,dt,step,U_statique)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))

Comparons pour $N=3001$:

In [ ]:
N = 2001
alpha = 0.6
Re = 10000
t_max = 0.025
dt = 0.005 # 0.025/0.005 = 5 pas de temps
step = 10000 # pas de svd pour le test

#numpy
start = time.time()
st,t_vec = svd_t.svd_scipy_RK4(Re,alpha,N,t_max,dt,step,U_statique)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))

#tensorflow
start = time.time()
st,t_vec = svd_t_tf.svd_tf_RK4(Re,alpha,N,t_max,dt,step,U_statique)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))

Nous remarquons que le temps mis par le code tensorflow se rapproche de celui du code numpy et s'équivalent pour des grandes matrices ($N\geq 1000$). Les deux vitesses sont d'ailleurs suspicieusement proches et pourtant nous pouvons vérifier avec les commandes "top" et "nvidia-smi" qu'il s'agit bien du cpu qui travaille lorsque le code numpy tourne et que le gpu travaille pour le code tensorflow.

## Implémentation sur gpu en C avec magma

Le code se trouve en annexe, dans le dossier "svd_gpu_c". Les résultats pour la valeur singulière maximale en fonction du temps sont renvoyés dans des fichiers .dat que nous analysons ci-dessous avec python grâce au module "pandas". 

Le fichier que nous analysons ci-dessous est le résultat de l'analyse par svd de l'écoulement caractérisé par $Re = 10000, \alpha=0.6$ et discrétisé avec $N=401$.

In [ ]:
import pandas

In [ ]:
dataSVD = pandas.read_csv('svd_gpu_c/data/data-Re=10000-alpha=0.6-N=401-dt=0.1.csv')
[t,sMax] = numpy.transpose(dataSVD.as_matrix(['Temps', 'ValSingMax']))

In [ ]:
pyplot.figure(figsize=(8,6));
pyplot.plot(t, sMax, '.');
pyplot.title('Valeur singulière maximale\n au cours du temps');
pyplot.xlabel('$t$');
pyplot.ylabel('Valeur singulière maximale');
pyplot.grid();
pyplot.show();

Nous vérifions ci-dessous la cohérence des calculs en c et en python en comparant les résultats pour les valeurs singulières au même instant.

In [ ]:
print("En python sur cpu: ")
print(numpy.transpose(t_vec[-5:]))
print(st[-5:,0])
print("En C sur gpu: ")
print(t[-5:])
print(sMax[-5:])

## Comparaison des vitesses d'exécution des implémentations sur cpu et gpu

L'environnement dans lequel sont effectués les test est le suivant:
- Magma 2.2.0
- Intel MKL 2018
- Cuda 8.0.61
- g++ 7.2.0
- Xubuntu 17.10

La carte graphique est la NVIDIA GeForce 920M.
Le cpu utilisé est un Intel Core i5-5200U, composé de deux coeurs physiques et pouvant exécuter 4 threads.

Notons que le code numpy utilise les quatre threads disponibles alors que le code magma n'utilise qu'un cpu thread à côté du gpu.

Le tableau suivant résume les différents tests réalisés sur cette machine.

| N | Re | alpha | tMax | dt | step | c/magma (gpu) | python/numpy (cpu) |
|---|----|-------|------|----|------|---------------|--------------------|
| 201|10000|0.6|100|0.1|10| 1m32s| 0m43s|
| 401|10000|0.6|100|0.1|10| 6m35s| 5m13s|
| 401|10000|0.6|  3|0.1| 5| 0m13s|  0m9s|
|2001|10000|0.6|  3|0.1| 5|17m16s| 9m25s|
|2001|10000|0.6|0.3|0.1| 1|  2m4s| 1m10s|
|2001|10000|0.6|0.3|0.1|10| 1m35s| 0m48s|
|3001|10000|0.6|0.3|0.1|10|  5m5s| 2m54s|
|4001|10000|0.6|0.3|0.1|10|out of memory| 6m19s|

Nous pouvons conclure que telle qu'elle est implémentée, la méthode ne s'exécute pas plus rapidement sur la carte graphique. Au contraire, elle est en moyenne deux fois plus lente avec le code pour gpu. 

Notons que lors des tests, $100\%$ du cpu thread est utilisé par magma. Le cpu pourrait donc être le composant limitant la vitesse dans notre cas, il serait donc intéressant de faire en sorte que le code magma utilise tous les coeurs disponibles sur la machine. Le cas $N=401$ semble s'exécuter avec des vitesses similaires en magma et numpy et pourrait donc être plus rapide en magma si celui-ci pouvait utiliser tous les coeurs.

$$\ $$
$$\ $$
$$\ $$

*La cellule qui suit permet l'importation immédiate de tout les fichiers externes nécessaires pour ce notebook.*

In [ ]:
import numpy,scipy,scipy.linalg,time,pandas
import matplotlib.pyplot as pyplot
from matplotlib import rcParams, cm
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

import operateurs
import operateurs_t
import svd
import svd_division_recursive as rd
import svd_t
import svd_t_tf